In [13]:
import requests
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import *
import matplotlib.pyplot as plt

In [14]:
set100_request = pd.read_html('https://en.wikipedia.org/wiki/SET50_Index_and_SET100_Index')
set100_stock = set100_request[0]['SET Symbol'].unique().tolist()
set100_stock_symbol = []
for symbol in range(len(set100_stock)):
    companie = set100_stock[symbol]
    set100_stock_symbol.append(str(companie) + ".BK")

In [15]:
# Calculate start_date to be 10 years ago
end_date = date.today()
start_date = end_date-pd.DateOffset(365*10)

# Download all symbols prices and stack them into a Multi-Index
prices_df = yf.download(tickers=set100_stock_symbol, start=start_date, end=end_date)
prices_df_stacked = prices_df.stack()
prices_df_stacked["Rolling_Return"] = prices_df_stacked.groupby(level=1).transform(lambda x: x.pct_change(252))['Close']
prices_df_stacked.index.names = ['Date', 'Symbol']
prices_df_stacked

[*********************100%***********************]  50 of 50 completed


Adj Close       Close        High         Low  \
Date       Symbol                                                      
2013-05-20 ADVANC.BK  190.170425  296.000000  305.000000  294.000000   
           AOT.BK      14.724933   16.700001   16.700001   16.200001   
           BANPU.BK    14.959231   24.774462   24.849993   24.547867   
           BBL.BK     157.203201  221.000000  224.000000  219.000000   
           BDMS.BK     14.735048   17.450001   17.650000   17.350000   
...                          ...         ...         ...         ...   
2023-05-15 TISCO.BK    93.750000   93.750000   93.750000   92.500000   
           TOP.BK      46.750000   46.750000   47.500000   46.500000   
           TRUE.BK      8.400000    8.400000    8.400000    8.400000   
           TTB.BK       1.490000    1.490000    1.500000    1.480000   
           TU.BK       14.900000   14.900000   15.100000   14.800000   

                            Open       Volume  Rolling_Return  
Date       Symbol                                              
2013-05-20 ADVANC.BK  298.000000    5592800.0             NaN  
           AOT.BK      16.299999   23288000.0             NaN  
           BANPU.BK    24.774462    8011185.0             NaN  
           BBL.BK     224.000000    4548200.0             NaN  
           BDMS.BK     17.549999   14469000.0             NaN  
...                          ...          ...             ...  
2023-05-15 TISCO.BK    93.000000    5572300.0       -0.053030  
           TOP.BK      47.500000    4378400.0       -0.126168  
           TRUE.BK      8.400000          0.0        0.735537  
           TTB.BK       1.500000  413260500.0        0.155039  
           TU.BK       15.000000   15228900.0       -0.113095  

[105000 rows x 7 columns]

In [16]:
# Aggregate On The Yearly And Symbol Level And Get The Last Return For The Given Year For Each Company, Then Rank The Companies By Their Yearly Return.
prices_df_stacked['Yearly_Return'] = prices_df_stacked.groupby([pd.Grouper(level='Date'), 'Symbol'])['Rolling_Return'].last()
prices_df_stacked['Rank'] = prices_df_stacked.groupby('Date')['Rolling_Return'].rank(ascending=False)

# Filter Only Top 5 Companies Per Momentum Rank At The End Of Each Year.
filtered_df = prices_df_stacked[prices_df_stacked['Rank']<5][:'2022']

# Move The Index With 1 Day In The Future To At The Begining Of Next Year For Convenience.
filtered_df = filtered_df.reset_index(level=1)
filtered_df.index = filtered_df.index + pd.DateOffset(1)
filtered_df = filtered_df.reset_index().set_index(['Date', 'Symbol']).unstack().stack()
filtered_df["Yearly_Return","Rank"]

KeyError: ('Yearly_Return', 'Rank')